In [48]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [49]:
import sys
sys.path.append('../src')

from models.classification_methods import get_classification_report

In [50]:
test_results_path = '../reports/test_results/'

list_df_t = os.listdir(test_results_path)
list_df_t.sort()
list_df_t

['DummyClassifier_bo_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_bo_users_Stance_test_results.csv',
 'DummyClassifier_bo_users_Timeline_test_results.csv',
 'DummyClassifier_cl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_cl_users_Stance_test_results.csv',
 'DummyClassifier_cl_users_Timeline_test_results.csv',
 'DummyClassifier_co_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_co_users_Stance_test_results.csv',
 'DummyClassifier_co_users_Timeline_test_results.csv',
 'DummyClassifier_gl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_gl_users_Stance_test_results.csv',
 'DummyClassifier_gl_users_Timeline_test_results.csv',
 'DummyClassifier_ig_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_ig_users_Stance_test_results.csv',
 'DummyClassifier_ig_users_Timeline_test_results.csv',
 'DummyClassifier_lu_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_lu_users_Stance_test_r

In [51]:
# Target list
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
]

dict_cp = {
    'cl':'Hydrox.',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}

names = list(dict_cp.values())
names

['Hydrox.', 'Lula', 'Sinovac', 'Church', 'Globo TV', 'Bolsonaro']

## Create complete table

In [52]:
# (vectorizer,estimator, path_sring) 
results_tuples_stance = [
    # Stance
    ("Stance", "-" ,"dummy", "DummyClassifier_{target}_users_Stance_test_results.csv"),
    ("Stance", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv"),
    ("Stance", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_users_emb_Stance_test_results.csv"),
    ("Stance", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv"),
    ("Stance", "-",  "llama3:7b zero-shot", "llama3_{target}_Stance_prompt2_Stance_test_results.csv"),
    
    # Texts
    ("Texts", "-" ,"dummy", "DummyClassifier_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_top_mentioned_timelines_emb_Texts_test_results.csv"),
    ("Texts", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Texts_test_results.csv"),
    ("Texts", "-" ,"bertabaporu-base (R)", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_scored_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [5] (R)", "llama3_{target}_filtered_Texts5_prompt2_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [10] (R)", "llama3_{target}_filtered_Texts10_prompt2_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [15] (R)", "llama3_{target}_filtered_Texts15_prompt2_Texts_test_results.csv"),
    
    # Timeline
    ("Timeline", "-" ,"dummy", "DummyClassifier_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_users_emb_Timeline_test_results.csv"),
    ("Timeline", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Timeline_test_results.csv"),
    ("Timeline", "-" ,"bertabaporu-base (R)", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_scored_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [5] (R)", "llama3_{target}_filteredTimeline5_prompt2_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [10] (R)", "llama3_{target}_filteredTimeline10_prompt2_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [15] (R)", "llama3_{target}_filteredTimeline15_prompt2_Timeline_test_results.csv"),
    
    ("Ensemble Texts + Timeline", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Texts_Timeline_test_results.csv"),
    ("Ensemble Texts + Stance", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Texts_Stance_test_results.csv"),
    ("Ensemble Stance + Timeline", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Stance_Timeline_test_results.csv"),
    ("Ensemble Stance + Timeline + Texts", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Stance_Timeline_Texts_test_results.csv")
]

list_results = []
for text_col, vectorizer, estimator, path_results in results_tuples_stance:
    
    list_cr = []
    
    for target in target_list:
        
        
        path = test_results_path + path_results.format(target = target)
        df_results = pd.read_csv(path)
        df_results_or = df_results.copy()
        
        # get classification report df
        df_classification_report = get_classification_report(df_results.test, df_results.pred, cr_args = {})
        
        # create multindex
        column_indexes = [(metric,dict_cp[target]) for metric in df_classification_report.columns]
        multi_index_cols = pd.MultiIndex.from_tuples(column_indexes, names=['metric', 'target'])
        rows_indexes = [(text_col, vectorizer, estimator, cl) for cl in df_classification_report.index]
        multi_index_rows = pd.MultiIndex.from_tuples(rows_indexes, names=['text_col','vectorizer', 'estimator', 'class'])
        df_classification_report.columns = multi_index_cols
        df_classification_report.index = multi_index_rows
        
        # print(text_col, vectorizer, estimator,target)
        # print(path)
        # display(df_classification_report)
        
        list_cr.append(df_classification_report)
        
    df_results = pd.concat(list_cr, axis = 1)
    
    list_results.append(df_results)
    
df_results_final = pd.concat(list_results)

In [53]:
df_results_final

metric                                                                        precision  \
target                                                                           Church   
text_col                           vectorizer estimator          class                    
Stance                             -          dummy              against       0.565943   
                                                                 accuracy      0.565943   
                                                                 weighted avg  0.320292   
                                                                 macro avg     0.282972   
                                                                 for           0.000000   
...                                                                                 ...   
Ensemble Stance + Timeline + Texts -          LogisticRegression against       0.762570   
                                                                 accuracy      0.747913   
                                                                 weighted avg  0.746758   
                                                                 macro avg     0.744355   
                                                                 for           0.726141   

metric                                                                           recall  \
target                                                                           Church   
text_col                           vectorizer estimator          class                    
Stance                             -          dummy              against       1.000000   
                                                                 accuracy      0.565943   
                                                                 weighted avg  0.565943   
                                                                 macro avg     0.500000   
                                                                 for           0.000000   
...                                                                                 ...   
Ensemble Stance + Timeline + Texts -          LogisticRegression against       0.805310   
                                                                 accuracy      0.747913   
                                                                 weighted avg  0.747913   
                                                                 macro avg     0.739193   
                                                                 for           0.673077   

metric                                                                         f1-score  \
target                                                                           Church   
text_col                           vectorizer estimator          class                    
Stance                             -          dummy              against       0.722814   
                                                                 accuracy      0.565943   
                                                                 weighted avg  0.409072   
                                                                 macro avg     0.361407   
                                                                 for           0.000000   
...                                                                                 ...   
Ensemble Stance + Timeline + Texts -          LogisticRegression against       0.783357   
                                                                 accuracy      0.747913   
                                                                 weighted avg  0.746569   
                                                                 macro avg     0.740980   
                                                                 for           0.698603   

metric                                                                            support  \
target                                                                            

In [54]:
df_results_final.to_excel("../reports/table_complete_results.xlsx")

## Create table f1 macro

In [55]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "macro avg" in col else False for col in df_results_final.index]

f1_macro_df = df_results_final.loc[mask_macro,mask_f1]
f1_macro_df[('f1-score','overall')] = f1_macro_df.mean(axis=1)

f1_macro_df

metric                                                                                      f1-score  \
target                                                                                        Church   
text_col                           vectorizer       estimator                    class                 
Stance                             -                dummy                        macro avg  0.361407   
                                   tf-idf           xgb                          macro avg  0.702021   
                                   bertabaporu-base xgb                          macro avg  0.853801   
                                   -                bertabaporu-base             macro avg  0.866276   
                                                    llama3:7b zero-shot          macro avg  0.729458   
Texts                              -                dummy                        macro avg  0.361407   
                                   tf-idf           xgb                          macro avg  0.596115   
                                   bertabaporu-base xgb                          macro avg  0.594754   
                                   -                bertabaporu-base             macro avg  0.586611   
                                                    bertabaporu-base (R)         macro avg  0.583417   
                                                    llama3:7b zero-shot [5] (R)  macro avg  0.499052   
                                                    llama3:7b zero-shot [10] (R) macro avg  0.561274   
                                                    llama3:7b zero-shot [15] (R) macro avg  0.503966   
Timeline                           -                dummy                        macro avg  0.361407   
                                   tf-idf           xgb                          macro avg  0.699122   
                                   bertabaporu-base xgb                          macro avg  0.661773   
                                   -                bertabaporu-base             macro avg  0.652707   
                                                    bertabaporu-base (R)         macro avg  0.701990   
                                                    llama3:7b zero-shot [5] (R)  macro avg  0.605633   
                                                    llama3:7b zero-shot [10] (R) macro avg  0.563199   
                                                    llama3:7b zero-shot [15] (R) macro avg  0.598183   
Ensemble Texts + Timeline          -                LogisticRegression           macro avg  0.698596   
Ensemble Texts + Stance            -                LogisticRegression           macro avg  0.820220   
Ensemble Stance + Timeline         -                LogisticRegression           macro avg  0.781126   
Ensemble Stance + Timeline + Texts -                LogisticRegression           macro avg  0.740980   

metric                                                                                                \
target                                                                                     Bolsonaro   
text_col                           vectorizer       estimator                    class                 
Stance                             -                dummy                        macro avg  0.462857   
                                   tf-idf           xgb                          macro avg  0.595312   
                                   bertabaporu-base xgb                          macro avg  0.625289   
                                   -                bertabaporu-base             macro avg  0.737297   
                                                    llama3:7b zero-shot          macro avg  0.462857   
Texts                              -                dummy                        macro avg  0.462857   
                                   tf-idf           xgb                          macro avg  0.501220   
                                   bertabaporu-

### Table for docs

In [56]:
f1_report = f1_macro_df.copy()
f1_report

metric                                                                                      f1-score  \
target                                                                                        Church   
text_col                           vectorizer       estimator                    class                 
Stance                             -                dummy                        macro avg  0.361407   
                                   tf-idf           xgb                          macro avg  0.702021   
                                   bertabaporu-base xgb                          macro avg  0.853801   
                                   -                bertabaporu-base             macro avg  0.866276   
                                                    llama3:7b zero-shot          macro avg  0.729458   
Texts                              -                dummy                        macro avg  0.361407   
                                   tf-idf           xgb                          macro avg  0.596115   
                                   bertabaporu-base xgb                          macro avg  0.594754   
                                   -                bertabaporu-base             macro avg  0.586611   
                                                    bertabaporu-base (R)         macro avg  0.583417   
                                                    llama3:7b zero-shot [5] (R)  macro avg  0.499052   
                                                    llama3:7b zero-shot [10] (R) macro avg  0.561274   
                                                    llama3:7b zero-shot [15] (R) macro avg  0.503966   
Timeline                           -                dummy                        macro avg  0.361407   
                                   tf-idf           xgb                          macro avg  0.699122   
                                   bertabaporu-base xgb                          macro avg  0.661773   
                                   -                bertabaporu-base             macro avg  0.652707   
                                                    bertabaporu-base (R)         macro avg  0.701990   
                                                    llama3:7b zero-shot [5] (R)  macro avg  0.605633   
                                                    llama3:7b zero-shot [10] (R) macro avg  0.563199   
                                                    llama3:7b zero-shot [15] (R) macro avg  0.598183   
Ensemble Texts + Timeline          -                LogisticRegression           macro avg  0.698596   
Ensemble Texts + Stance            -                LogisticRegression           macro avg  0.820220   
Ensemble Stance + Timeline         -                LogisticRegression           macro avg  0.781126   
Ensemble Stance + Timeline + Texts -                LogisticRegression           macro avg  0.740980   

metric                                                                                                \
target                                                                                     Bolsonaro   
text_col                           vectorizer       estimator                    class                 
Stance                             -                dummy                        macro avg  0.462857   
                                   tf-idf           xgb                          macro avg  0.595312   
                                   bertabaporu-base xgb                          macro avg  0.625289   
                                   -                bertabaporu-base             macro avg  0.737297   
                                                    llama3:7b zero-shot          macro avg  0.462857   
Texts                              -                dummy                        macro avg  0.462857   
                                   tf-idf           xgb                          macro avg  0.501220   
                                   bertabaporu-

In [57]:
f1_report.reset_index(drop=False, inplace=True)
f1_report

metric                            text_col        vectorizer  \
target                                                         
0                                   Stance                 -   
1                                   Stance            tf-idf   
2                                   Stance  bertabaporu-base   
3                                   Stance                 -   
4                                   Stance                 -   
5                                    Texts                 -   
6                                    Texts            tf-idf   
7                                    Texts  bertabaporu-base   
8                                    Texts                 -   
9                                    Texts                 -   
10                                   Texts                 -   
11                                   Texts                 -   
12                                   Texts                 -   
13                                Timeline                 -   
14                                Timeline            tf-idf   
15                                Timeline  bertabaporu-base   
16                                Timeline                 -   
17                                Timeline                 -   
18                                Timeline                 -   
19                                Timeline                 -   
20                                Timeline                 -   
21               Ensemble Texts + Timeline                 -   
22                 Ensemble Texts + Stance                 -   
23              Ensemble Stance + Timeline                 -   
24      Ensemble Stance + Timeline + Texts                 -   

metric                     estimator      class  f1-score                      \
target                                             Church Bolsonaro   Hydrox.   
0                              dummy  macro avg  0.361407  0.462857  0.334878   
1                                xgb  macro avg  0.702021  0.595312  0.734366   
2                                xgb  macro avg  0.853801  0.625289  0.830948   
3                   bertabaporu-base  macro avg  0.866276  0.737297  0.848144   
4                llama3:7b zero-shot  macro avg  0.729458  0.462857  0.638889   
5                              dummy  macro avg  0.361407  0.462857  0.334878   
6                                xgb  macro avg  0.596115  0.501220  0.603727   
7                                xgb  macro avg  0.594754  0.495578  0.609524   
8                   bertabaporu-base  macro avg  0.586611  0.462857  0.446504   
9               bertabaporu-base (R)  macro avg  0.583417  0.462857  0.570531   
10       llama3:7b zero-shot [5] (R)  macro avg  0.499052  0.296142  0.469796   
11      llama3:7b zero-shot [10] (R)  macro avg  0.561274  0.328245  0.440347   
12      llama3:7b zero-shot [15] (R)  macro avg  0.503966  0.285253  0.416946   
13                             dummy  macro avg  0.361407  0.462857  0.334878   
14                               xgb  macro avg  0.699122  0.728220  0.898944   
15                               xgb  macro avg  0.661773  0.692262  0.776979   
16                  bertabaporu-base  macro avg  0.652707  0.767310  0.809799   
17              bertabaporu-base (R)  macro avg  0.701990  0.822060  0.872170   
18       llama3:7b zero-shot [5] (R)  macro avg  0.605633  0.588752  0.639513   
19      llama3:7b zero-shot [10] (R)  macro avg  0.563199  0.611771  0.609199   
20      llama3:7b zero-shot [15] (R)  macro avg  0.598183  0.586106  0.611868   
21                LogisticRegression  macro avg  0.698596  0.728220  0.898944   
22                LogisticRegression  macro avg  0.820220  0.800537  0.811570   
23                LogisticRegression  macro avg  0.781126  0.767310  0.905882   
24                LogisticRegression  macro avg  0.740980  0.642395  0.846643   

metric                                          
target   Sinovac  Globo TV      Lu

In [58]:
f1_report.columns

MultiIndex([(  'text_col',          ''),
            ('vectorizer',          ''),
            ( 'estimator',          ''),
            (     'class',          ''),
            (  'f1-score',    'Church'),
            (  'f1-score', 'Bolsonaro'),
            (  'f1-score',   'Hydrox.'),
            (  'f1-score',   'Sinovac'),
            (  'f1-score',  'Globo TV'),
            (  'f1-score',      'Lula'),
            (  'f1-score',   'overall')],
           names=['metric', 'target'])

In [59]:
new_columns = [col[0] if col[1] == '' else col[1] for col in f1_report.columns]
new_columns

['text_col',
 'vectorizer',
 'estimator',
 'class',
 'Church',
 'Bolsonaro',
 'Hydrox.',
 'Sinovac',
 'Globo TV',
 'Lula',
 'overall']

In [60]:
f1_report.columns = new_columns
f1_report

,text_col,vectorizer,estimator,class,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,-,dummy,macro avg,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf,xgb,macro avg,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base,xgb,macro avg,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,-,bertabaporu-base,macro avg,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,-,llama3:7b zero-shot,macro avg,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,-,dummy,macro avg,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf,xgb,macro avg,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base,xgb,macro avg,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,-,bertabaporu-base,macro avg,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,-,bertabaporu-base (R),macro avg,0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [61]:
f1_report.drop(['class'],axis = 1, inplace=True)
f1_report

,text_col,vectorizer,estimator,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,-,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf,xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base,xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,-,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,-,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,-,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf,xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base,xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,-,bertabaporu-base,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,-,bertabaporu-base (R),0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [62]:
f1_report.insert(
    1, 
    "classifier", 
    f1_report.apply(
        lambda x: f"{x['vectorizer']} + {x['estimator']}" if x['vectorizer'] != '-' else x['estimator'],
        axis = 1
        ).to_list()

)
f1_report.drop(['estimator', 'vectorizer'],axis =1, inplace = True)
f1_report

,text_col,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf + xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base + xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,bertabaporu-base,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,bertabaporu-base (R),0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [63]:
f1_report.rename({"text_col":"input"}, axis = 1, inplace=True)
f1_report

,input,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf + xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base + xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,bertabaporu-base,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,bertabaporu-base (R),0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [64]:
#f1_report.set_index(['input'],inplace=True)
#f1_report.drop('input', axis = 1,inplace=True)

In [65]:
def generate_latex_with_multirow_and_bold(df):
    latex_code = ''
    latex_code += '\\begin{table}[H]'
    latex_code += "\\begin{tabular}{ll|rrrrrrr}\n\\toprule\n"
    latex_code += "input & classifier & Church & Bolsonaro & Hydrox. & Sinovac & Globo TV & Lula & overall \\\\ \n\\midrule\n"

    last_input = None
    multirow_count = 0

    for input_value in df['input'].unique():
        subset = df[df['input'] == input_value]
        max_overall_idx = subset['overall'].idxmax()

        for i, row in subset.iterrows():
            if row['input'] == last_input:
                latex_code += "& "
                multirow_count += 1
            else:
                if multirow_count > 0:
                    latex_code = latex_code.replace(f"multirow{{{multirow_count}}}", f"multirow{{{multirow_count + 1}}}", 1)
                if last_input is not None:
                    latex_code += "\\cmidrule(lr){1-9}\n"
                latex_code += f"\\multirow{{{1}}}{{*}}{{{row['input']}}} & "
                multirow_count = 1

            if i == max_overall_idx:
                row_data = [f"\\textbf{{{row[col]:.2f}}}" if col not in ['input', 'classifier'] else f"\\textbf{{{row[col]}}}" for col in df.columns[1:]]
                latex_code += " & ".join(row_data) + " \\\\ \n"
            else:
                latex_code += " & ".join([f"{row[col]:.2f}" if isinstance(row[col], float) else str(row[col]) for col in df.columns[1:]]) + " \\\\ \n"
            
            last_input = row['input']

    if multirow_count > 0:
        latex_code = latex_code.replace(f"multirow{{{multirow_count}}}", f"multirow{{{multirow_count + 1}}}", 1)

    latex_code += "\\cmidrule(lr){1-9}\n"
    latex_code += "\\bottomrule\n\\end{tabular}"
    latex_code += "\caption{F1 macro results}"
    latex_code += '\label{table:results_f1_macro}\n'
    latex_code += '\end{table}'

    return latex_code

In [66]:
f1_report.input = f1_report.input.map({
    "Stance": "S",
    "Timeline": 'UT',
    "Texts": 'UFT',
    "Ensemble Texts + Timeline": 'E1',
    
    "Ensemble Texts + Stance":'E2',
    "Ensemble Stance + Timeline":'E3',
    "Ensemble Stance + Timeline + Texts":'E4'
    
})

In [67]:
f1_report

,input,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,S,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,S,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,S,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,S,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,S,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,UFT,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,UFT,tf-idf + xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,UFT,bertabaporu-base + xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,UFT,bertabaporu-base,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,UFT,bertabaporu-base (R),0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [68]:
str_latex = generate_latex_with_multirow_and_bold(f1_report)

In [69]:
print(str_latex)

\begin{table}[H]\begin{tabular}{ll|rrrrrrr}
\toprule
input & classifier & Church & Bolsonaro & Hydrox. & Sinovac & Globo TV & Lula & overall \\ 
\midrule
\multirow{2}{*}{S} & dummy & 0.36 & 0.46 & 0.33 & 0.35 & 0.37 & 0.34 & 0.37 \\ 
& tf-idf + xgb & 0.70 & 0.60 & 0.73 & 0.75 & 0.67 & 0.65 & 0.68 \\ 
& bertabaporu-base + xgb & 0.85 & 0.63 & 0.83 & 0.81 & 0.78 & 0.77 & 0.78 \\ 
& \textbf{bertabaporu-base} & \textbf{0.87} & \textbf{0.74} & \textbf{0.85} & \textbf{0.83} & \textbf{0.86} & \textbf{0.79} & \textbf{0.82} \\ 
& llama3:7b zero-shot & 0.73 & 0.46 & 0.64 & 0.58 & 0.77 & 0.70 & 0.65 \\ 
\cmidrule(lr){1-9}
\multirow{2}{*}{UFT} & dummy & 0.36 & 0.46 & 0.33 & 0.35 & 0.37 & 0.34 & 0.37 \\ 
& \textbf{tf-idf + xgb} & \textbf{0.60} & \textbf{0.50} & \textbf{0.60} & \textbf{0.67} & \textbf{0.55} & \textbf{0.58} & \textbf{0.58} \\ 
& bertabaporu-base + xgb & 0.59 & 0.50 & 0.61 & 0.66 & 0.53 & 0.57 & 0.58 \\ 
& bertabaporu-base & 0.59 & 0.46 & 0.45 & 0.61 & 0.51 & 0.48 & 0.52 \\ 
& bertabap